# Creación de la gran base de datos
En este Jupyter Notebook nos encargaremos de modificar todas las bases de datos de la carpeta "datos_crudos" para transformarlas a un formato común, y así generar un archivo que contendrá toda la información que utilizaremos en el análisis final. El proceso se dividirá en los siguientes pasos:

1. Revisar los datos y su formato, para determinar si es necesario realizar alguna conversión o modificación antes de utilizarlos.
2. Corregir los datos que no sigan la estructura común, transformándolos a un formato estandarizado.
3. Unir todas las bases de datos y escribir un archivo que contenga todos los datos crudos consolidados.

In [1]:
#librerias necesarias para esta parte
import pandas as pd
import seaborn as sns
import pyarrow.parquet as pq
import pyarrow as pa

# nos situamos en la carpeta corespondiente
%cd "C:\Users\Jose\Documents\GitHub\imt2200---actividad-2024\datos de economia\datos_crudos"

C:\Users\Jose\Documents\GitHub\imt2200---actividad-2024\datos de economia\datos_crudos


## 1. Primeramente revisaremos todos los archivos pertenecientes a la careta "datos_crudos"

### Abriremos el archivo que contiene la informacion del ingreso mediano real

In [5]:
df_imr = pd.read_excel("imr general.xlsx", engine='openpyxl')
df_imr

,NaN,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Unit of measurement,NaN,Index,NaN
2,Month,NaN,NaN,NaN
3,2018 January,NaN,NaN,109.38
4,2018 February,NaN,NaN,108.97
...,...,...,...,...
76,2024 February,NaN,NaN,158.13
77,2024 March,NaN,(p),159.19
78,Data extracted on 07 Oct 2024 23:28 UTC (GMT) ...,NaN,NaN,NaN
79,Legend:,NaN,NaN,NaN


Esta base de datos parece ser bastante mal, bamos hacerle un par de retoques

In [6]:
# existen algunas filas que tienen el nombre de las columnas, los eliminaremos ya que no son necesarias
df_imr_fix = df_imr.loc[3:77]

# al existir una columna de valor nulo, consigo las columnas para axceder mediante indexcacion
columnas_imr = list(df_imr_fix.columns)
mes_imr = pd.DataFrame({"moth": df_imr_fix[columnas_imr[0]].tolist()})

# luego de crear el dataframe nuevo que tiene la misma informacion que la columna nula, lo concateno y filtro las columnas que tienen informacion
df_imr_fix = mes_imr.merge(right= df_imr_fix, left_on="moth", right_on=columnas_imr[0])
df_imr_fix = df_imr_fix[['moth', 'Unnamed: 3']]

# por ultimo renombro la columna Unnamed: 3 y guardo el dataframe
df_imr_fix = df_imr_fix.rename(columns={'Unnamed: 3': 'ingreso mediano real'})
df_imr_fix

,moth,ingreso mediano real
0,2018 January,109.38
1,2018 February,108.97
2,2018 March,109.53
3,2018 April,109.84
4,2018 May,109.69
...,...,...
70,2023 November,155.08
71,2023 December,156.32
72,2024 January,158.21
73,2024 February,158.13


In [34]:
df_imr_fix.to_json(r"C:\Users\Jose\Documents\GitHub\imt2200---actividad-2024\datos de economia\datos_ordenados\imr general.json", orient="columns")

### abirmos el archivo que tiene la imformacion sobre ingreso mediano real por ocupacion

In [2]:
# abrimos el archivo y vemos la info que trae
df_imr_ocupacion = pd.read_csv("imr por ocupacion.csv")
df_imr_ocupacion.info()

# devido a que columnas que necesito son el mes, grupo de ocupacion y el valor los filtrare y tambien cambiare la columna grupo de ocupacion para  
df_imr_ocupacion = df_imr_ocupacion[['Month', 'Occupational group', 'Value']]
df_imr_ocupacion['Occupational group'] = df_imr_ocupacion['Occupational group'].astype("category")
df_imr_ocupacion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DTI_CL_UNIDAD_MEDIDA  1350 non-null   object 
 1   Unit of measurement   1350 non-null   object 
 2   DTI_CL_MES            1350 non-null   object 
 3   Month                 1350 non-null   object 
 4   DTI_CL_GRUPO_OCU_IR   1350 non-null   object 
 5   Occupational group    1350 non-null   object 
 6   Value                 1350 non-null   float64
 7   Flag Codes            18 non-null     object 
 8   Flags                 18 non-null     object 
dtypes: float64(1), object(8)
memory usage: 95.1+ KB


,Month,Occupational group,Value
0,2018 January,Miembros del poder ejecutivo y de los cuerpos ...,112.57
1,2018 January,Profesionales científicos e intelectuales,107.64
2,2018 January,Técnicos y profesionales de nivel medio,109.18
3,2018 January,Empleados de oficina,108.73
4,2018 January,"Trabajadores de servicios personales, protecci...",111.58
...,...,...,...
1345,2024 March,"Trabajadores de servicios personales, protecci...",9.30
1346,2024 March,"Vendedores, promotores y modelos",8.50
1347,2024 March,"Oficiales, operarios y artesanos de artes mecá...",4.90
1348,2024 March,Operadores de instalaciones y máquinas y monta...,7.20


In [38]:
df_imr_ocupacion.to_json(r"C:\Users\Jose\Documents\GitHub\imt2200---actividad-2024\datos de economia\datos_ordenados\imr por categoria.json", orient="columns")

### abrimos el archivo que contiene el indice mediano real por seccion economica

In [11]:
df_imr_seccion = pd.read_excel("imr por seccion economica.xlsx", header=3, index_col="Mining")
df_imr_seccion

ValueError: Index Mining invalid (sheet: 0)